# <center> Where are the restaurants in Montréal ? </center>
<br>

<img src='https://meetings.mtl.org/sites/meetings/files/styles/hero/public/2018-03/33751.jpg?itok=Dg4m-lW3' width="800" height="800">

Made by : Thierry Lévesque 
<br>
July 16th, 2021

## Introduction

Whether you are visiting Montréal or are interested in opening a restaurant, it is good to know where the current restaurants are. Montréal has a lot to offer in terms of nightlife, cafés and restaurant. But where to start if you don't know the city ?

As part of the IBM Data Science Certification, students like me have to do a capstone project leveraging the Foursquare API. The API makes it possible to know a lot of information about venues around a point of interest. We can therefore make a list of the venues in the Montréal neighborhoods and make maps and charts to show where to go to find what you are looking for.

To make this project possible, I used a list of postal codes and the associated neighborhood found on this wikipedia page : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H. With these informations, I was able to find the coordinates of each neighborhood, which allowed me to create a map of Montréal and it's neighborhoods. I also used the coordinates to find venues in each neighborhoods using the Foursquare API.

Once I have the informations about the kind of venues found in each neighborhoods, I can use a clustering algorithm to make clusters of similar neighborhoods.

## Get the data

In [1]:
import pandas as pd
import numpy as np
import requests
import pgeocode
import folium
import matplotlib.pyplot as plt

### Get the postal codes from wikipedia

In [2]:
postal_codes_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H'
montreal_wiki = pd.read_html(postal_codes_url)[0]

In [3]:
montreal_wiki

,0,1,2,3,4,5,6,7,8
0,H0ANot assigned,H1APointe-aux-Trembles,"H2ASaint-Michel,East",H3ADowntown Montreal North(McGill University),H4ANotre-Dame-de-GrâceNortheast,H5APlace Bonaventure,H7ADuvernay-Est,H8ANot assigned,H9ADollard-des-OrmeauxNorthwest
1,H0BNot assigned,H1BMontreal East,H2BAhuntsicNorth,H3BDowntown MontrealEast,H4BNotre-Dame-de-GrâceSouthwest,H5BPlace Desjardins,H7BSaint-François,H8BNot assigned,H9BDollard-des-OrmeauxEast
2,H0CNot assigned,H1CRivière-des-PrairiesNortheast,H2CAhuntsicCentral,H3CGriffintown(Includes Île Notre-Dame & Île S...,H4CSaint-Henri,H5CNot assigned,H7CSaint-Vincent-de-Paul,H8CNot assigned,H9CL'Île-BizardNortheast
3,H0ENot assigned,H1ERivière-des-PrairiesSouthwest,H2EVillerayNortheast,H3EL'Île-Des-Soeurs,H4EVille Émard,H5ENot assigned,H7EDuvernay,H8ENot assigned,H9EL'Île-BizardSouthwest
4,H0GNot assigned,H1GMontréal-NordNorth,H2GPetite-PatrieNortheast,H3GDowntown MontrealSoutheast (Concordia Unive...,H4GVerdunNorth,H5GNot assigned,H7GPont-Viau,H8GNot assigned,H9GDollard-des-OrmeauxSouthwest
5,H0HReserved0H0: Santa Claus,H1HMontréal-NordSouth,H2HPlateau Mont-RoyalNorth,H3HDowntown MontrealSouthwest,H4HVerdunSouth,H5HNot assigned,H7HAuteuilWest,H8HNot assigned,H9HSainte-Geneviève / Pierrefonds
6,H0JNot assigned,H1JAnjouWest,H2JPlateau Mont-RoyalNorth Central,H3JPetite-Bourgogne,H4JCartiervilleCentral,H5JNot assigned,H7JAuteuilNortheast,H8JNot assigned,H9JKirkland
7,H0KNot assigned,H1KAnjouEast,H2KCentre-SudNorth,H3KPointe-Saint-Charles,H4KCartiervilleSouthwest,H5KNot assigned,H7KAuteuilSouth,H8KNot assigned,H9KSenneville
8,H0LNot assigned,H1LMercierNorth,H2LCentre-SudSouth,H3LAhuntsicSouthwest,H4LSaint-LaurentInner Northeast,H5LNot assigned,H7LSainte-Rose,H8LNot assigned,H9LNot assigned
9,H0MAkwesasne Region1A0: Akwesasne,H1MMercierWest,H2MAhuntsicEast,H3MCartiervilleNortheast,H4MSaint-LaurentEast,H5MNot assigned,H7MVimont,H8MNot assigned,H9MNot assigned


In [4]:
update_dict = {}
montreal_df = pd.DataFrame()
replace_dict = {'Akwesasne Region1A0: ':'', 'North':'','South':'','East':'','West':''}

for col in range(len(montreal_wiki.columns)):
    for row in range(len(montreal_wiki)):
        text = montreal_wiki.loc[row,col]
        if 'Not assigned' in text:
            continue
        if 'H0H' in text[0:3]:
            continue
        update_dict['PostalCode'] = text[0:3]
        update_dict['Neighborhood'] = text[3:].replace('Akwesasne Region1A0: ','').replace('North','').replace('South','').replace('East','').replace('east','').replace('West','').replace('west','').replace(',','')
        montreal_df = montreal_df.append(update_dict, ignore_index=True)

In [5]:
pd.set_option('display.max_rows', None)
montreal_df

,Neighborhood,PostalCode
0,Akwesasne,H0M
1,Pointe-aux-Trembles,H1A
2,Montreal,H1B
3,Rivière-des-Prairies,H1C
4,Rivière-des-Prairies,H1E
5,Montréal-Nord,H1G
6,Montréal-Nord,H1H
7,Anjou,H1J
8,Anjou,H1K
9,Mercier,H1L


In [6]:
montreal_df.shape

(122, 2)

### Get the coordinates

In [7]:
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
update_coords_dict = {}
montreal_coords_df = pd.DataFrame()

for i, PostalCode in enumerate(montreal_df['PostalCode']):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(PostalCode)
    
    if not g.empty:
        #print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        update_coords_dict['Latitude'] = g.latitude
        update_coords_dict['Longitude'] = g.longitude
        update_coords_dict['PostalCode'] = PostalCode
        montreal_coords_df = montreal_coords_df.append(update_coords_dict, ignore_index=True)
        
montreal_coords_df

,Latitude,Longitude,PostalCode
0,45.6986,-73.5025,H0M
1,45.6753,-73.5016,H1A
2,45.6320,-73.5075,H1B
3,45.6656,-73.5367,H1C
4,45.6342,-73.5842,H1E
5,45.6109,-73.6211,H1G
6,45.5899,-73.6389,H1H
7,45.6097,-73.5794,H1J
8,45.6097,-73.5472,H1K
9,45.6043,-73.5178,H1L


In [8]:
montreal_df = montreal_df.join(montreal_coords_df.set_index('PostalCode'), on='PostalCode')
montreal_df

,Neighborhood,PostalCode,Latitude,Longitude
0,Akwesasne,H0M,45.6986,-73.5025
1,Pointe-aux-Trembles,H1A,45.6753,-73.5016
2,Montreal,H1B,45.6320,-73.5075
3,Rivière-des-Prairies,H1C,45.6656,-73.5367
4,Rivière-des-Prairies,H1E,45.6342,-73.5842
5,Montréal-Nord,H1G,45.6109,-73.6211
6,Montréal-Nord,H1H,45.5899,-73.6389
7,Anjou,H1J,45.6097,-73.5794
8,Anjou,H1K,45.6097,-73.5472
9,Mercier,H1L,45.6043,-73.5178


### Now that we have the coordinates of each neighborhoods, we can make a map of Montréal

In [9]:
latitude, longitude = 45.508888, -73.561668

# create map of Manhattan using latitude and longitude values
map_montreal = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(montreal_df['Latitude'], montreal_df['Longitude'], montreal_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_montreal)  
    
map_montreal

In [10]:
CLIENT_ID = 'PV5DTIJMZJ24IJVHMI5IEKAOE0YMCA5RUCCLB0TJMYEK2LLC'
CLIENT_SECRET = 'Z4IJRDRSAUB3FJTBX0FXPATRLAH5NZHJX5L3NSJG4TJI5EO0'
ACCESS_TOKEN = 'L1CUPPTSIJQOKYSWBVW2Y1KTYWMFTVRDDR2Q4BUAZKXS3EU2'
VERSION = '20210715' # Foursquare API version
LIMIT = 500 # A default Foursquare API limit value

### The Foursquare API make it possible to know the venues around a point of interest

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
montreal_venues = getNearbyVenues(montreal_df['Neighborhood'], montreal_df['Latitude'], montreal_df['Longitude'])

In [13]:
montreal_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akwesasne,45.6986,-73.5025,Site Historique National De La Bataille De La ...,45.699499,-73.505149,Historic Site
1,Pointe-aux-Trembles,45.6753,-73.5016,Parc-nature de la Pointe-aux-Prairies,45.678834,-73.501162,Park
2,Pointe-aux-Trembles,45.6753,-73.5016,AMT Gare Pointe-aux-Trembles,45.674882,-73.504908,Train Station
3,Pointe-aux-Trembles,45.6753,-73.5016,Parc Yves-Thériault,45.678675,-73.502037,Park
4,Montreal,45.6320,-73.5075,Mini Excavation Et Deneigement Quinn Inc,45.631120,-73.505353,Construction & Landscaping
5,Montreal,45.6320,-73.5075,Amsal Inc,45.634212,-73.506707,Clothing Store
6,Montreal,45.6320,-73.5075,Tapis Du Manufacturier En Gros,45.634454,-73.507054,Furniture / Home Store
7,Montreal,45.6320,-73.5075,Simple IP inc.,45.632263,-73.512558,Electronics Store
8,Rivière-des-Prairies,45.6656,-73.5367,STM Arrêt #61273,45.665108,-73.535263,Bus Stop
9,Rivière-des-Prairies,45.6656,-73.5367,Toitures D L MAT,45.663916,-73.535549,Construction & Landscaping
